In [1]:
import torch
from datasets import DatasetUCI
from envs import LalEnvFirstAccuracy
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
import math
import numpy as np
import random
from replay_buffer import ReplayBuffer
from dqn import DQN

/home/lazaioan/anaconda3/envs/RLATorch/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


TO DO

1. CHANGE THE CODE ACCORDING TO THE SITE: https://unnatsingh.medium.com/deep-q-network-with-pytorch-d1ca6f40bfda

2. USE LLAVA TO ASK THE QUESTION FROM STACK OVERFLOW AND UPDATE THE ORIGINAL CODE IN TENSORFLOW!!!!

Setup and initialisation.

Parameters for dataset and model.

In [2]:
WARM_START_EPISODES = 2 # Number of warm-start episodes to create transitions for the Replay Buffer.
STATE_LENGTH = 30 # Size of the state data.
TRAIN_SET_SIZE = -1 # Use -1 for using all data points.
SUBSET = -1 # Use -1 for using all data points, 0 for even, 1 for odd.
N_JOBS = 1 # Can set more if we want to parallelise.
# Remove the dataset that will be used for testing.
# ['australian', 'breast_cancer', 'diabetis', 'flare_solar', 'german', 'heart', 'mushrooms', 'waveform', 'wdbc']
# possible_dataset_names = ['breast_cancer', 'diabetis', 'flare_solar', 'german', 'heart', 'mushrooms', 'waveform', 'wdbc']
possible_dataset_names = ['waveform']
test_dataset_names = ['diabetis']
# The quality is measured according to a given quality measure "quality_method". 
QUALITY_METHOD = metrics.accuracy_score
BUFFER_SIZE = 10000 # Size of the Replay Buffer.
BATCH_SIZE = 32 # Size of the Minibatch size for the Replay Buffer.

Initialise the datasets.

In [3]:
dataset = DatasetUCI(possible_dataset_names, n_state_estimation=STATE_LENGTH, subset=SUBSET, size=TRAIN_SET_SIZE)
# If we want to measure test error along with training.
dataset_test = DatasetUCI(test_dataset_names, n_state_estimation=STATE_LENGTH, subset=SUBSET, size=TRAIN_SET_SIZE)

Build the classifier.

In [4]:
model = LogisticRegression(n_jobs=N_JOBS)

Initialise the environment.

In [5]:
env = LalEnvFirstAccuracy(dataset, model, quality_method=QUALITY_METHOD)
env_test = LalEnvFirstAccuracy(dataset_test, model, quality_method=QUALITY_METHOD)

Initialise the Replay Buffer.

In [6]:
replay_buffer = ReplayBuffer(Buffer_Size=BUFFER_SIZE, Batch_Size=BATCH_SIZE)

Warm - start episodes for the Replay Buffer.

In [7]:
# Keep track of episode duration to compute average.
episode_durations = []
episode_number = 1

for _ in range(WARM_START_EPISODES):
    
    print("Episode {}.".format(episode_number))
    print("\n")
    # Reset the environment to start a new episode.
    # The classifier_state contains vector representation of state of the environment (depends on the classifier).
    # The _action contains vector representations of all actions available to be taken at the next step.
    state, _action, reward = env.reset()
    batch = 6
    done = False
    episode_duration = 6

    # Before we reach a done state, make steps.
    while not done:
        # Choose a random action.
        batch = random.randint(1,40)
        if batch > env.n_actions:
            done = True
            break
        # Getting numbers from 0 to n_actions.
        print("n.actions", env.n_actions)
        inputNumbers =range(0,env.n_actions)
        # Non-repeating using sample() function.
        a = np.array(random.sample(inputNumbers, batch))
        action = _action[:,a]
        _state, _action, reward, done = env.step(a)

        # Store the transition in the replay buffer.
        replay_buffer.store_transition(state, action, reward, _state, _action, done)
        # Get ready for next step.
        state = _state
        episode_duration += batch
    print("Episode {}. | Budget size is {}.".format(episode_number, episode_duration))
    episode_number += 1
    episode_durations.append(episode_duration)

# Compute the average episode duration of episodes generated during the warm start procedure.
av_episode_duration = np.mean(episode_durations)
BIAS_INITIALIZATION = -av_episode_duration/2
print(BIAS_INITIALIZATION)

Episode 1.


n.actions 2463
n.actions 2440
n.actions 2426
Final achieved ACC is 0.7952.
Episode 1. | Budget size is 49.
Episode 2.


n.actions 2463
n.actions 2446
n.actions 2406
n.actions 2380
n.actions 2377
n.actions 2340
n.actions 2325
n.actions 2323
n.actions 2299
n.actions 2259
n.actions 2226
n.actions 2204
n.actions 2177
n.actions 2154
n.actions 2134
n.actions 2122
n.actions 2114
n.actions 2084
n.actions 2076
n.actions 2074
n.actions 2049
n.actions 2014
n.actions 2000
n.actions 1987
n.actions 1953
n.actions 1948
n.actions 1934
n.actions 1932
n.actions 1928
n.actions 1903
n.actions 1889
n.actions 1884
n.actions 1856
n.actions 1823
n.actions 1795
n.actions 1788
n.actions 1782
n.actions 1745
n.actions 1716
n.actions 1683
n.actions 1662
n.actions 1623
n.actions 1590
n.actions 1564
n.actions 1556
n.actions 1538
n.actions 1515
n.actions 1483
n.actions 1457
n.actions 1443
n.actions 1423
n.actions 1394
n.actions 1357
n.actions 1328
n.actions 1297
n.actions 1274
n.actions 1260
n.actions 12

In [8]:
from collections import namedtuple, deque
experiences = namedtuple("Experience", field_names=["state","action","reward","next_state","next_action","done"])
memory = deque(maxlen=10000)
e = experiences(state,action,reward,_state,_action,done)
memory.append(e)
memory


deque([Experience(state=array([0.07465001, 0.10851647, 0.14206922, 0.24471618, 0.25059533,
              0.29880774, 0.30115846, 0.38222654, 0.47412465, 0.47518037,
              0.68852595, 0.69115732, 0.73617578, 0.7600859 , 0.83331704,
              0.9097671 , 0.98157631, 0.98723959, 0.9913819 , 0.99756386,
              0.99888689, 0.99969446, 0.99981465, 0.99988567, 0.99993366,
              0.99994739, 0.99995234, 0.99995889, 0.99998219, 0.99999356]), action=array([[0.916672  , 0.99866922, 0.14422618, 0.99574503, 0.99371512,
               0.06563122, 0.28867899, 0.37025928, 0.45938251, 0.9937422 ,
               0.99998599, 0.21083801, 0.6498622 , 0.99926106, 0.99829996,
               0.09334048, 0.38587836, 0.98309289, 0.99994908, 0.5922601 ,
               0.77571492, 0.96807914, 0.99684493, 0.95150298, 0.87072432,
               0.10835619, 0.16162364],
              [0.98343337, 0.98699572, 0.98710812, 0.9879072 , 0.98931017,
               0.99934199, 0.99904492, 0.986095

#### Parameters for training RL.

In [9]:
DIRNAME = './agents/1-australian-logreg-8-to-1/' # The resulting agent of this experiment will be written in a file.

# Replay buffer parameters.
REPLAY_BUFFER_SIZE = 1e4
PRIOROTIZED_REPLAY_EXPONENT = 3

# Agent parameters.
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
TARGET_COPY_FACTOR = 0.01
BIAS_INITIALIZATION = 0 # Default 0 # will be set to minus half of average duration during warm start experiments.

# Warm start parameters.
NN_UPDATES_PER_WARM_START = 100

# Episode simulation parameters.
EPSILON_START = 1
EPSILON_END = 0.1
EPSILON_STEPS = 1000

# Training parameters.
TRAINING_ITERATIONS = 10 # Reduce for test.
TRAINING_EPISODES_PER_ITERATION = 5 # At each training iteration x episodes are simulated.
NN_UPDATES_PER_ITERATION = 5 # At each training iteration x gradient steps are made.

# Validation and test parameters.
N_VALIDATION = 2 # Reduce for test.
N_TEST = 2 # Reduce for test.
VALIDATION_TEST_FREQUENCY = 3 # Every x iterations, val and test are performed.

Initialize the DQN agent.

In [10]:
agent = DQN(State_Length=STATE_LENGTH)

In [13]:
minibatch = replay_buffer.minibatch()
print(minibatch)

tensor([[0.0095, 0.0426, 0.0454, 0.2300, 0.2411, 0.2469, 0.3681, 0.4190, 0.7247,
         0.8494, 0.9109, 0.9468, 0.9830, 0.9831, 0.9870, 0.9895, 0.9941, 0.9963,
         0.9964, 0.9966, 0.9971, 0.9975, 0.9981, 0.9983, 0.9989, 0.9990, 0.9995,
         0.9997, 0.9997, 0.9999],
        [0.0654, 0.0925, 0.1642, 0.2194, 0.2484, 0.2761, 0.3502, 0.4157, 0.4752,
         0.4830, 0.7113, 0.7163, 0.7369, 0.7710, 0.8164, 0.9192, 0.9830, 0.9897,
         0.9913, 0.9980, 0.9990, 0.9997, 0.9998, 0.9999, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000],
        [0.0314, 0.1054, 0.1508, 0.1822, 0.2394, 0.2548, 0.2821, 0.3182, 0.3305,
         0.4779, 0.5683, 0.7048, 0.7162, 0.7806, 0.8740, 0.8753, 0.8811, 0.9055,
         0.9877, 0.9901, 0.9938, 0.9969, 0.9983, 0.9983, 0.9996, 0.9997, 0.9997,
         0.9998, 0.9999, 0.9999],
        [0.0876, 0.0963, 0.1075, 0.1306, 0.1409, 0.1485, 0.2370, 0.2763, 0.3617,
         0.3629, 0.7140, 0.7174, 0.8510, 0.8662, 0.8929, 0.9461, 0.9822, 0.9880,
       

##### Do updates of the network based on warm start episodes.

In [12]:
for _ in range(NN_UPDATES_PER_WARM_START):
    # Sample a batch from the replay buffer proportionally to the probability of sampling.
    print("Compute the minibatch.", "\n")
    minibatch = replay_buffer.sample()

    # Use batch to train an agent. Keep track of temporal difference errors during training.
    print("Train the agent using the minibatch.", "\n")
    td_error = agent.train(minibatch)

    # Update probabilities of sampling each datapoint proportionally to the error.
    print("Update errors.")
    replay_buffer.update_td_errors(td_error, minibatch.indices)

Compute the minibatch. 



AttributeError: 'ReplayBuffer' object has no attribute 'sample'

# NEOS KWDIKAS!!!!!! https://unnatsingh.medium.com/deep-q-network-with-pytorch-d1ca6f40bfda

# Train RL

##### Run multiple training iterations. Each iteration consists of:
- generating episodes following agent's actions with exploration,
- validation and test episodes for evaluating performance,
- Q-network updates.

In [ ]:
train_episode_rewards = []
i_episode = 0

In [ ]:
is_training = True
for iteration in range(TRAINING_ITERATIONS):

    print("ITERATION {}.".format(iteration+1))
    # GENERATE NEW EPISODES
    # Compute epsilon value according to the schedule.
    epsilon = max(EPSILON_END, EPSILON_START-iteration*(EPSILON_START-EPSILON_END)/EPSILON_STEPS)

    # Simulate training episodes.
    episode_number = 1
    
    episode_duration = 6

    for _ in range(TRAINING_EPISODES_PER_ITERATION):

        # Reset the environment to start a new episode.
        classifier_state, _action, reward = env.reset()
        batch = 6
        done = False
        episode_duration = 6
        done = False

        # Keep track of stats of episode to analyse it in tensorboard.
        episode_reward = 0
        
        # Run an episode.
        while not done:
            train_batch = batch
            if (train_batch-2 >= env.n_actions) or (train_batch+2 >= env.n_actions) or (train_batch >= env.n_actions):
                done = True
            else:
                train_batch = env._find_batch_size(train_batch, reward, env.n_actions)
                episode_duration += train_batch
                action = agent.get_action(classifier_state, _action, train_batch)

                if np.random.ranf() < epsilon:
                    inputNumbers =range(0,env.n_actions)
                    action = np.array(random.sample(inputNumbers, train_batch))

                # With epsilon probability, take a random action.
                # action is a vector that corresponds to a taken action.
                action = _action[:,action]

                # Make another step.
                _state, _action, reward, done = env.step(action)

                # Store a step in replay buffer.
                replay_buffer.store_transition(classifier_state, 
                                            action, 
                                            reward, 
                                            _state, 
                                            _action, 
                                            done)
                
                # Change a state of environment.
                classifier_state = _state
        print("Iteration {}. | Episode {}. | Budget size is {}.".format(iteration, episode_number, episode_duration))
        episode_number += 1

    # VALIDATION AND TEST EPISODES.
    episode_summary = tf.Summary()
    if iteration%VALIDATION_TEST_FREQUENCY == 0:

        # Validation episodes are run. Use env for it.
        print("\n\n\n")
        print("Iteration {}. Validation episodes.".format(iteration+1))
        all_durations_validation = []
        for i in range(N_VALIDATION):
            done = False
            state, _action, reward = env.reset()
            validation_batch = batch
            all_durations_validation.append(validation_batch)
            while not(done):
                if (validation_batch-2 >= env.n_actions) or (validation_batch+2 >= env.n_actions) or (validation_batch >= env.n_actions):
                    done = True
                else:
                    validation_batch = env._find_batch_size(validation_batch, reward, env.n_actions)
                    all_durations_validation.append(validation_batch)
                    action = policy_rl(agent, state, _action, validation_batch)        
                    action = _action[:,action]
                    next_state, _action, reward, done = env.step(action)
                    state = next_state
        
        # Test episodes are run. Use env_test for it.
        print("\n\n\n")
        print("Iteration {}. Test episodes.".format(iteration+1))
        all_durations_test = []
        for i in range(N_TEST):
            done = False
            state, _action, reward = env_test.reset()
            test_batch = batch
            all_durations_test.append(test_batch)
            while not(done):
                if (test_batch-2 >= env_test.n_actions) or (test_batch+2 >= env_test.n_actions) or (test_batch >= env_test.n_actions):
                    done = True
                else:
                    test_batch = env_test._find_batch_size(test_batch, reward, env_test.n_actions)
                    all_durations_test.append(test_batch)
                    action = policy_rl(agent, state, _action, test_batch)        
                    action = _action[:,action]
                    next_state, _action, reward, done = env_test.step(action)
                    state = next_state
   
    # NEURAL NETWORK UPDATES.
    for _ in range(NN_UPDATES_PER_ITERATION):
        minibatch = replay_buffer.sample_minibatch(BATCH_SIZE)
        td_error = agent.train(minibatch)
        replay_buffer.update_td_errors(td_error, minibatch.indices)

In [ ]:
env.episode_qualities

#### To see the results in tensorboard:

On the server:
tensorboard --logdir=./

Or:
tensorboard --logdir ./  --port 6006  --host 0.0.0.0

On the computer:
ssh -N -f -L localhost:6006:localhost:6006 konyushk@iccvlabsrv20.iccluster.epfl.ch && open http://localhost:6006